## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). 
You don't need this line when you write your own C++ programs. 
I need it to set my [interactive compiler (Cling)](https://root.cern/cling/). 
For your own program, use [CMake](https://cmake.org/).

In [2]:
#include "includeLibraries.h"

## Header inclusion for C++

In [3]:
#include <iostream> // For cout and cerr
#include <sstream> // For formatting error messages
#include <memory> // For smart pointer
#include <stdexcept> // For runtime exceptions
#include <opencv2/opencv.hpp> // For OpenCV
#include <FlyCapture2.h> // For the Flea3 camera

## Add the namespaces

In [4]:
using namespace FlyCapture2;

In [5]:
using namespace cv;

In [6]:
using namespace std;

## Open the camera

In [9]:
//-----------------------------------
PGRGuid getFlea3Id(int aCameraId = 0)
//-----------------------------------
{
    PGRGuid guid;
    BusManager bus_manager;
    unsigned int number_of_cameras;
    
    // Retrieve the number of cameras
    FlyCapture2::Error error_status = bus_manager.GetNumOfCameras(&number_of_cameras);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }
    
    // The camera exists
    if (number_of_cameras > aCameraId)
    {
        error_status = bus_manager.GetCameraFromIndex(aCameraId, &guid);
        if (error_status != PGRERROR_OK)
        {
            throw runtime_error(error_status.GetDescription());
        }
    }
    // The camera does not exist
    else
    {
        stringstream error_message;
        error_message << "FATAL ERROR" << endl; 
        error_message << "Number of cameras detected: " << number_of_cameras << endl;
        error_message << "Camera ID " << aCameraId << " does not exist" << endl;
        error_message << "The program will terminate.";
        throw runtime_error(error_message.str());
    }
    
    return guid;
}

In [11]:
//------------------------------------------------------
void openFlea3Camera(Camera& aCamera, int aCameraId = 0)
//------------------------------------------------------
{
    // Get the ID
    PGRGuid guid = getFlea3Id(aCameraId);
    
    // Connect to a camera
    FlyCapture2::Error error_status = aCamera.Connect(&guid);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }
    
    // Get the camera configuration
    FC2Config config;
    error_status = aCamera.GetConfiguration(&config);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Set the number of driver buffers used to 10.
    config.numBuffers = 10;

    // Set the camera configuration
    error_status = aCamera.SetConfiguration(&config);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Start capturing images
    error_status = aCamera.StartCapture();
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }
}

In [12]:
//--------------------------------------
void releaseFlea3Camera(Camera& aCamera)
//--------------------------------------
{
    // Stop capturing images
    FlyCapture2::Error error_status = aCamera.StopCapture();
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Disconnect the camera
    error_status = aCamera.Disconnect();
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }
}

In [17]:
Mat getFrame(Camera& aCamera)
{
    // Retrieve an image
    Image rawImage;
    FlyCapture2::Error error_status = aCamera.RetrieveBuffer(&rawImage);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Convert the raw image
    Image convertedImage;
    error_status = rawImage.Convert(PIXEL_FORMAT_MONO8, &convertedImage);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Convert to OpenCV
    return Mat(convertedImage.GetRows(), convertedImage.GetCols(), CV_8UC1, convertedImage.GetData());
}

In [20]:
Camera camera;
openFlea3Camera(camera, 0);

Standard Exception: FATAL ERROR
Number of cameras detected: 0
Camera ID 0 does not exist
The program will terminate.

## Display the images from the camera

We create a window to display the images from the webcam

In [ ]:
namedWindow("Webcam", WINDOW_GUI_EXPANDED); // Create a window

We display the images in a loop

In [ ]:
while (waitKey(1) != 27) // Exit when pressing <ESC>
{
    // Grab a new frame
    Mat frame;
    video_input >> frame;

    // Make sure everything went well
    if (frame.empty())
    {
        video_input.release(); // We are now done with the camera, stop it
        throw runtime_error("OpenCV cannot grab a new frame from the camera, the program will terminate");
    }
       
    // Display the image
    imshow("Webcam", frame);
}

Release the camera

In [23]:
releaseFlea3Camera(camera);

Standard Exception: Generation count is incorrect.

We don't need the window any more, destroy it

In [ ]:
destroyAllWindows(); // Destroy all the created windows